Teraz, mając rozkład $M^TM = \bar{U} \Sigma \bar{U}^{-1}$, trzeba się zastanowić co dalej -- jak zrobić z tego SVD macierzy $M$?

Niech $M = X\Lambda Y^T$ będzie szukanym SVD. Wiemy, że:
1. $\Lambda$ jest diagonalna i na diagonali ma pierwiastki z elementów diagonali $\Sigma$.
2. kolumny $Y$ są wektorami własnymi $M^TM$, czyli $Y = \bar{U}$. Jak teraz wyznaczyć $X$?
3. skoro znamy $M$, to mając równość $M = X\Lambda Y^T$ mamy $X = M(\Lambda Y^T)^{-1} = MY^{-T}\Lambda^{-1} = MY\Lambda^{-1}$.

Wówczas, skoro $M = P_m C P_m^T$, to $C = P_m ^T M P_m = P_m ^T X \Lambda Y^T P_m$, czyli:
$$B = \begin{bmatrix}
U_1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & U_2
\end{bmatrix}P_m ^T X \Lambda Y^T P_m\begin{bmatrix}
V_1 ^T & 0\\
0 & V_2 ^T
\end{bmatrix}$$

# SVD dla bidiagonalnej działa!
Teraz trzeba nauczyć się sprowadzać dowolną macierz $A$ do postaci bidiagonalnej

In [131]:
import numpy as np
from numpy.linalg import norm

In [253]:
np.random.seed(0)
A = np.random.rand(4, 4)
M, N = A.shape

U0T = np.eye(M)
V0 = np.eye(N)

B = np.copy(A)
print(f"Przed wejściem do pętli: B=\n{B}\n")
for i in range(N):
    
    print(f"KROK {i}\n")
    
    # działanie w kolumnie
    u = np.copy(B[i:, i])
    k = len(u)
    u[0] += norm(u)
    v = (np.zeros_like(u) if np.isclose(norm(u), 0) else u/norm(u))
    Hi = np.eye(k) - 2 * v.reshape((k, 1)) @ v.reshape((1, k))
    Hleft = np.block([[np.eye(i), np.zeros((i, k))],
                      [np.zeros((k, i)), Hi]])
    
    print(f"Hleft=\n{Hleft}\n")
    
    U0T = (Hleft @ U0T)

    print(f"Hleft * B=\n{Hleft @ B}\n")
    print(f"U0T * A=\n{U0T @ A}\n")
    
    # B[i:, i] = Hi @ B[i:, i]
    B = (Hleft @ B)
    
    if i == N-1:
        break
    
    # działanie w wierszu
    u = np.copy(B[i, (i+1):])
    k = len(u)
    u[0] += norm(u)
    v = (np.zeros_like(u) if np.isclose(norm(u), 0) else u/norm(u))
    Hi = np.eye(k) - 2 * v.reshape((k, 1)) @ v.reshape((1, k))
    Hright = np.block([[np.eye(i+1), np.zeros((i+1, k))],
                [np.zeros((k, i+1)), Hi]])
    
    print(f"Hright=\n{Hright}\n")
    
    V0 = (V0 @ Hright)
    
    print(f"B * Hright=\n{B @ Hright}\n")
    B = (B @ Hright)
    print(f"A * V0=\n{A @ V0}\n")
    
    # B[i, (i+1):] = B[i, (i+1):] @ Hi

Przed wejściem do pętli: B=
[[0.5488135  0.71518937 0.60276338 0.54488318]
 [0.4236548  0.64589411 0.43758721 0.891773  ]
 [0.96366276 0.38344152 0.79172504 0.52889492]
 [0.56804456 0.92559664 0.07103606 0.0871293 ]]

KROK 0

Hleft=
[[-0.41701418 -0.32191274 -0.73223605 -0.43162684]
 [-0.32191274  0.9268689  -0.16634704 -0.0980556 ]
 [-0.73223605 -0.16634704  0.62162013 -0.22304133]
 [-0.43162684 -0.0980556  -0.22304133  0.86852515]]

Hleft * B=
[[-1.31605478e+00 -1.18644770e+00 -1.00261646e+00 -9.39180375e-01]
 [-1.16416224e-16  2.13886300e-01  7.28821667e-02  5.54628199e-01]
 [-3.48076506e-16 -5.99221353e-01 -3.78481678e-02 -2.37988616e-01]
 [-1.52929760e-16  3.46352194e-01 -4.17967530e-01 -3.64920981e-01]]

U0T * A=
[[-1.31605478e+00 -1.18644770e+00 -1.00261646e+00 -9.39180375e-01]
 [-1.16416224e-16  2.13886300e-01  7.28821667e-02  5.54628199e-01]
 [-3.48076506e-16 -5.99221353e-01 -3.78481678e-02 -2.37988616e-01]
 [-1.52929760e-16  3.46352194e-01 -4.17967530e-01 -3.64920981e-01]]

H

In [254]:
mybidiag = U0T @ A @ V0
np.allclose(mybidiag, B)

True

Sprawdzamy ostateczny kod:

In [258]:
from importlib import reload
import bidiagonalizacja
reload(bidiagonalizacja)
from bidiagonalizacja import bidiagonalize

A = np.random.rand(7, 4)
A

array([[0.16983042, 0.8781425 , 0.09834683, 0.42110763],
       [0.95788953, 0.53316528, 0.69187711, 0.31551563],
       [0.68650093, 0.83462567, 0.01828828, 0.75014431],
       [0.98886109, 0.74816565, 0.28044399, 0.78927933],
       [0.10322601, 0.44789353, 0.9085955 , 0.29361415],
       [0.28777534, 0.13002857, 0.01936696, 0.67883553],
       [0.21162812, 0.26554666, 0.49157316, 0.05336255]])

In [269]:
U0, B, V0T, transposed = bidiagonalize(A)
np.allclose(U0 @ B @ V0T, A)

True

In [271]:
B[np.where(np.abs(B) < 1e-15)] = 0
B

array([[-1.59178554, -1.93340143,  0.        ,  0.        ],
       [ 0.        , -1.09711237, -0.16842912,  0.        ],
       [ 0.        ,  0.        , -0.73190831, -0.52513328],
       [ 0.        ,  0.        ,  0.        , -0.6823574 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [2]:
import numpy as np
A = np.array([[1, 2, 3],
              [4, 5, 6]])
A

array([[1, 2, 3],
       [4, 5, 6]])

In [3]:
U, S, VT = np.linalg.svd(A)

In [6]:
VT

array([[-0.42866713, -0.56630692, -0.7039467 ],
       [ 0.80596391,  0.11238241, -0.58119908],
       [ 0.40824829, -0.81649658,  0.40824829]])